In [ ]:
from trace_generator import *
import pandas
import random
import numpy

In [ ]:
tree = construct_random_tree(4674, 6666, tree_parameters={'min': 3, 'max': 6, 'mode': 4})

In [ ]:
show_tree(tree)

In [ ]:
import time
tt = int(time.time())
tt2 = int(time.time())
print(tt)
print(tt2)

# numpy.random.seed(tt)
# random.seed(tt)
numpy.random.seed(1660384210)
random.seed(1660384210)

tree_with_sim_properties = convert_pt_to_list_and_attach_properties(tree, max_loop_depth=10)
tree_with_sim_properties

In [ ]:
numpy.random.seed(1660384210)
random.seed(1660384210)

activities_list = get_element_from_pt(tree_with_sim_properties, prefix='a_')

modelled_activities_list = generate_activity_simulation_values(
    activities_list,
    time_modifier_function=[lambda x: (0.5 / x) - 0.5, lambda x: numpy.power(x, 0.25) - 1],
    time_modifier_function_probability_distribution=[2/3, 1/3],
    # time_modifier_function_probability_distribution=None,
    cost_modifier_function=[lambda x: -0.05 * numpy.power(x, 1.25), lambda x: -0.15 * numpy.power(x, 1.5)],
    # cost_modifier_function=None,
    cost_modifier_function_probability_distribution=[4/5, 1/5]
)

wrapper_time_func = lambda rand_val: lambda x: 0.5 * x + numpy.random.uniform(low=0.01, high=0.1) if rand_val >= x else (x - rand_val)**2 + rand_val * 0.5 + numpy.random.uniform(low=0.01, high=0.1)
wrapper_cost_func = lambda rand_val: lambda x: -0.1 * numpy.sqrt(x) if rand_val >= x else -0.1 * numpy.sqrt(rand_val)

# Average resources
modelled_resources_list = generate_resources_simulation_values(
    activities_list,
    min_resources=2,
    max_resources=2,
    resources_limited_to_activities='s',
    apply_jack_of_all_trades_penalty=True,
    time_modifier_function=[wrapper_time_func(numpy.random.randint(low=3, high=7)) for i in range(0, len(activities_list))],
    cost_modifier_function=[wrapper_cost_func(numpy.random.randint(low=5, high=10)) for i in range(0, len(activities_list))]
)

# Resources which are better at time efficiency but worse for cost.
modelled_resources_list += generate_resources_simulation_values(
    activities_list,
    start_resource_iter=len(modelled_resources_list),
    min_resources=2,
    max_resources=2,
    min_time_base_modifier=-0.5,
    max_time_base_modifier=-0.5,
    min_cost_base_modifier=0.5,
    max_cost_base_modifier=0.5,
    apply_jack_of_all_trades_penalty=False,
    resources_limited_to_activities='n',
    time_modifier_function=[lambda x: 0.1 * numpy.sqrt(x)],
    cost_modifier_function=[lambda x: -0.2 * numpy.sqrt(x)]
)

# Resources which are better at cost efficiency but worse for time.
modelled_resources_list += generate_resources_simulation_values(
    activities_list,
    start_resource_iter=len(modelled_resources_list),
    min_resources=2,
    max_resources=2,
    min_time_base_modifier=0.5,
    max_time_base_modifier=0.5,
    min_cost_base_modifier=-0.5,
    max_cost_base_modifier=-0.5,
    apply_jack_of_all_trades_penalty=False,
    resources_limited_to_activities='n',
    time_modifier_function=[lambda x: 0.1 * x**2],
    cost_modifier_function=[lambda x: -0.01 * numpy.sqrt(x)]
)

traces = generate_traces(
    tree_with_sim_properties=tree_with_sim_properties,
    modelled_resources=modelled_resources_list,
    modelled_activities=modelled_activities_list
)

In [ ]:
pandas.DataFrame(traces).to_excel('model_pareto_trace.xlsx')